# Budowniczy

Wzorzec **budowniczy** służy do tworzenia złożonych obiektów krok po kroku. Oddziela proces tworzenia obiektu od jego reprezentacji, co pozwala na tworzenie różnych wariantów tego samego typu obiektu bez modyfikacji jego struktury.

Przeznaczenie i zastosowanie:
- używany w przypadkach, gdy tworzenie obiektu wymaga wielu kroków lub konfiguracji,
- stosowany w tworzeniu obiektów o wielu opcjonalnych parametrach, eliminując problem z rozbudowanymi konstruktorami lub inicjalizatorami,
- powszechne rozwiązanie w konfiguracji obiektów, tworzeniu dokumentów, generowaniu interfejsów użytkownika i budowaniu złożonych encji w bazach danych.

Wzorzec budowniczy skupia się na sposobie konstrukcji obiektu, a nie na obiektach składowych. Proces tworzenia obiektu jest wręcz często niezależny od mniejszych obiektów, z których się składa. Wzorzec ten używany jest wszędzie tam, gdzie trzeba przejąć kontrolę nad procesem tworzenia obiektu. Proces kończy się, gdy konkretny budowniczy skończy pracę.

Jednym z częstszych realnych zastosowań tego wzorca jest otrzymanie tego samego efektu na wiele sposobów:
- eksport do pliku graficznego w zależności od docelowego formatu,
- szyfrowanie tej samej informacji na wiele różnych sposobów,
- nałożenie filtru na obraz.

## Implementacja

Cel: zapis dwuwymiarowej listy do pliku:
- tsv (csv oddzielany spacjami) dla wartości numerycznych,
- csv dla wartości tekstowych.

In [27]:
numbers = [
    [1.234, 5.667, 4, 1.1],
    [0.87653, 1.09, 6.01, 7.5423],
    [3.986, 654.12, 8.098, 53.123],
]

In [28]:
texts = [
    ["ala", "ma", "kota"],
    ["kot", "ma", "ale"],
    ["siala", "baba", "mak"],
]

Implementacja każdego z budowniczych powinna spełniać ten sam kontrakt. Należy zatem przygotować generyczną klasę abstrakcyjną, która posłuży jako szablon dla każdej z klas definiujących budowniczych.

In [29]:
from abc import ABC, abstractmethod

In [30]:
class Builder(ABC):
    _data: list

    @abstractmethod
    def set_data(self, data: list) -> None:
        pass

    @abstractmethod
    def save_data(self, filename: str) -> None:
        pass

Implementacja każdego z budowniczych będzie polegała na utworzeniu osobnych klas, gdzie każda z nich będzie odpowiedzialna za zapis pliku w innym formacie.

In [31]:
class CSVBuilder(Builder):
    _data: list

    def set_data(self, data: list) -> None:
        self._data = [f"{','.join(row)}\n" for row in data]

    def save_data(self, filename: str) -> None:
        with open(filename, "w") as f:
            f.writelines(self._data)

In [32]:
class TSVBuilder(Builder):
    _data: list

    def set_data(self, data: list) -> None:
        self._data = [" ".join(map(str, row)) for row in data]

    def save_data(self, filename: str) -> None:
        with open(filename, "w") as f:
            for row in self._data:
                f.write(f"{row}\n")

Tworzenie obiektów na podstawie wielu parametrów może być nieprecyzyjne mając do czynienia z wieloma klasami budowniczych jednocześnie. Rozwiązaniem tego problemu jest osobna **klasa nadzorcy**, która, na podstawie szeregu czynników, samodzielnie dopasowuje klasę budowniczego tworzącą ostateczny obiekt.

In [38]:
class Director:
    _builder: Builder
    _data: list

    def set_data(self, data: list) -> None:
        self._data = data
        self._set_builder()

    def create_file(self) -> None:
        self._builder.set_data(self._data)
        self._builder.save_data("result.csv")

    def _set_builder(self) -> None:
        if isinstance(self._data[0][0], (int, float)):
            self._builder = TSVBuilder()
        else:
            self._builder = CSVBuilder()

Finalne uruchomienie

In [39]:
director = Director()
director.set_data(numbers)
director.create_file()

In [40]:
director.set_data(texts)
director.create_file()

## Podsumowanie

Wzorzec budowniczy z powodzeniem przejmuje proces tworzenia obiektów, w szczególności gdy ich ostateczna klasa zależy od zewnętrznych czynników. Takie podejście rodzi następujące konsekwencje:
- zwiększona kontrola nad procesem tworzenia obiektów,
- niewielkie ignorowanie obiektów składowych,
- każdy typ produktu wymaga nowego budowniczego,
- brak mamy pewności, że budowniczy zbuduje obiekt ze wszystkich składowych,
- brak pewności, że wszystkie atrybuty budowanego obiektu zostaną zainicjalizowane,
- znaczna objętość kodu.